In [ ]:
# from google.colab import drive
# drive.mount('/content/GoogleDrive')

In [1]:
!pip install split-folders

In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.models import Sequential, Model, load_model
from keras.applications import VGG16
from keras.layers import Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D, Dropout

from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, zero_one_loss, classification_report
from sklearn.model_selection import train_test_split
from imutils import paths

import matplotlib.pyplot as plt
import seaborn as sb
import splitfolders
import cv2
import random

In [3]:
inputFolder = 'C:\\Users\\mooha\\jupyter\\Graduation Project\\image_datasets\\Brain Cancer'
outputFolder = 'C:\\Users\\mooha\\jupyter\\Graduation Project\\image_dataset_(Brain)'
# inputFolder = '/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/Brain Cancer'
# outputFolder = '/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/image_dataset_(Brain) '

In [ ]:
splitfolders.ratio(inputFolder, outputFolder, seed = 44, ratio = (0.8, 0.0, 0.2))

Copying files: 20000 files [01:19, 251.00 files/s] 


In [4]:
imgPaths = list(paths.list_images(outputFolder))
print(imgPaths[0].split('\\'))

random.shuffle(imgPaths)

classes = [] # classes = ['brain_menin', 'brain_glioma', 'brain_pituitary', 'no_tumor']
for imgPath in imgPaths:
  className = imgPath.split('\\')[-2]

  if className not in classes:
    classes.append(className)

print(classes)

X_train = []
y_train = []

X_test = []
y_test = []

IMGSIZE = 128
img_names = []

for imgPath in imgPaths:
  trainOrTest = imgPath.split('\\')[-3]
  className = imgPath.split('\\')[-2]

  img = cv2.imread(imgPath) #cv2.IMREAD_GRAYSCALE
  img = cv2.resize(img, (IMGSIZE,IMGSIZE))

  if trainOrTest == 'train':
    X_train.append(img)
    y_train.append(classes.index(className))
    img_names.append(imgPath.split('\\')[-1])

  elif trainOrTest == 'test':
    X_test.append(img)
    y_test.append(classes.index(className))

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

['C:', 'Users', 'mooha', 'jupyter', 'Graduation Project', 'image_dataset_(Brain)', 'test', 'brain_glioma', 'brain_glioma_0001.jpg']
['brain_glioma', 'brain_menin', 'no_tumor', 'brain_pituitary']
16000 16000
4000 4000


In [ ]:
df = pd.DataFrame({
    'File Name' : img_names,
    'Category' : y_train #y
})

# print(classes.index('brain_menin'))
# print(classes.index('brain_glioma'))
# print(classes.index('brain_pituitary'))
# print(classes.index('no_tumor'))

print(classes)
df.head()

In [ ]:
sb.countplot(x = y_train)

In [ ]:
sb.countplot(x = y_test)

In [8]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [9]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(16000, 128, 128, 3) (16000,)
(4000, 128, 128, 3) (4000,)


In [17]:
input_shape = (IMGSIZE, IMGSIZE, 3)
num_classes = 4

# Load the pre-trained VGG16 model (without the top classification layer)
base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = input_shape)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

vgg_model = Sequential()

vgg_model.add(base_model)
vgg_model.add(MaxPooling2D(pool_size = (2, 2)))
vgg_model.add(Flatten())

vgg_model.add(Dense(128, activation = 'relu'))
vgg_model.add(Dropout(0.25))

vgg_model.add(Dense(64, activation = 'relu'))
vgg_model.add(Dense(num_classes, activation = 'softmax'))

vgg_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 3, restore_best_weights = True)
results = vgg_model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 5, batch_size = 32, callbacks = [early_stopping]) 
# val_loss, val_acc = vgg_model.evaluate(X_test, y_test)

y_pred = vgg_model.predict(X_test)

vgg_model.summary()

Epoch 1/5
500/500 [==============================] - 595s 1s/step - loss: 0.7912 - accuracy: 0.8150 - val_loss: 0.2438 - val_accuracy: 0.9055
Epoch 2/5
500/500 [==============================] - 587s 1s/step - loss: 0.2691 - accuracy: 0.9013 - val_loss: 0.1903 - val_accuracy: 0.9293
Epoch 3/5
500/500 [==============================] - 583s 1s/step - loss: 0.2131 - accuracy: 0.9229 - val_loss: 0.1593 - val_accuracy: 0.9388
Epoch 4/5
500/500 [==============================] - 577s 1s/step - loss: 0.1745 - accuracy: 0.9368 - val_loss: 0.1777 - val_accuracy: 0.9290
Epoch 5/5
125/125 [==============================] - 112s 898ms/step
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 2, 2, 512)        0         
 2D)       

In [ ]:
base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (IMGSIZE, IMGSIZE, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(128, activation = 'relu')(x)
x = Dropout(0.25)(x)

x = Dense(64, activation = 'relu')(x)
predictions = Dense(4, activation = 'softmax')(x)

model = Model(inputs = base_model.input, outputs = predictions)

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 3, restore_best_weights = True) 
results = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 32, callbacks = [early_stopping])  
# val_loss, val_acc = model.evaluate(X_test, y_test)

y_pred = model.predict(X_test)

model.summary()

In [ ]:
# model.save('C:\\Users\\mooha\\jupyter\\Graduation Project\\model_image_(VGG16).h5')
loaded_model = load_model('C:\\Users\\mooha\\jupyter\\Graduation Project\\model_image_(VGG16).h5')

# model.save('/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/model_image_(Brain).h5')
# loaded_model = load_model('/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/model_image_(Brain).h5')  

In [ ]:
plt.plot(results.history['accuracy'], label = 'Training Accuracy')
plt.plot(results.history['val_accuracy'], label = 'Testing Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(results.history['loss'], label = 'Training Loss')
plt.plot(results.history['val_loss'], label = 'Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [20]:
print(classes)
print(y_test[:25], '\n')
print(y_pred[:25], '\n')

predict = []
for i in range(len(y_pred)):
  predict.append(np.argmax(y_pred[i]))

print(predict[:25], '\n')

cm = confusion_matrix(y_test, predict)
print(cm, '\n')

zol = zero_one_loss(y_test, predict, normalize = False)
print(zol, '\n')

cr = classification_report(y_test, predict)
print(cr)

['brain_glioma', 'brain_menin', 'no_tumor', 'brain_pituitary']
[1 3 1 3 0 1 1 1 1 3 3 3 2 0 3 0 2 3 1 1 0 3 0 1 2] 

[[5.29886383e-05 4.98554967e-02 7.41263984e-09 9.50091600e-01]
 [2.06210156e-04 4.01468622e-03 4.07676271e-06 9.95775044e-01]
 [3.00027372e-04 9.99699831e-01 3.58079213e-12 1.49752665e-07]
 [7.09261816e-11 1.37761581e-05 3.09321492e-11 9.99986112e-01]
 [9.99281645e-01 6.86695625e-04 9.97236516e-09 3.17186132e-05]
 [2.70216972e-01 7.29706585e-01 1.66143451e-07 7.62886630e-05]
 [2.33221218e-01 7.61532426e-01 9.99403346e-05 5.14642848e-03]
 [1.61696628e-01 8.38230908e-01 1.82393251e-07 7.22345867e-05]
 [1.99519582e-06 9.99997914e-01 3.21204886e-17 6.65011587e-11]
 [3.00944247e-09 1.76915783e-04 3.22145111e-09 9.99823093e-01]
 [2.84871063e-03 4.53170808e-03 6.62837863e-07 9.92618918e-01]
 [1.68647304e-01 1.73639908e-01 2.18626689e-02 6.35850072e-01]
 [2.03777598e-18 1.08826386e-14 9.99999940e-01 5.63559078e-17]
 [9.99997199e-01 2.72518082e-06 3.59051303e-14 4.23547482e-12]
 

In [ ]:
plt.figure(figsize = (5,5))
ConfusionMatrixDisplay(cm).plot()
plt.show()

In [ ]:
sb.countplot(x = predict)

In [ ]:
plot_model(model, show_shapes = True)

In [ ]:
test_img_path = 'C:\\Users\\mooha\\jupyter\\Graduation Project\\image_dataset_(Brain)\\test\\brain_glioma\\brain_glioma_4736.jpg'
# test_img_path = '/content/GoogleDrive/MyDrive/Colab Notebooks/Neural Networks/Graduation Project/image_dataset_(Brain)/test/brain_glioma/brain_glioma_4736.jpg'  
test_img = cv2.imread(test_img_path)
test_img = cv2.resize(test_img, (IMGSIZE,IMGSIZE))

plt.imshow(test_img)

test_img = test_img[tf.newaxis, ...]
test_predict = vgg_model.predict(test_img)
# test_predict = loaded_model.predict(test_img)

str1 = test_img_path.split('\\')[-1].split('_')[0]
str2 = test_img_path.split('\\')[-1].split('_')[1]
str3 = classes[np.argmax(test_predict)].split('_')[0] + ' ' + classes[np.argmax(test_predict)].split('_')[1]

plt.title('Actual: ' + str1 + ' ' + str2 + '\n Predicted: ' + str3)

print(test_predict)
print(classes)
plt.show()